#intialization

import libraries

In [1]:
import pandas as pd
import numpy as np
import math
import random
from sklearn.utils.extmath import randomized_svd

#SVD

In [2]:
def svdd(X,k):
  U, Sigma, VT = randomized_svd(X, n_components=k)

# Reconstruct the original matrix using the truncated SVD components
  X_svd = U.dot(np.diag(Sigma)).dot(VT)

# Compare the reconstructed matrix with the original matrix
  return X_svd

#3DRT

min max normalization

In [3]:
def minmax_normalization(x, a=0, b=1):
    x_min = np.min(x)
    x_max = np.max(x)
    return (x - x_min) * (b - a) / (x_max - x_min) + a

Z score Normalization

In [4]:
def z_score_normalization(x):
    mean = np.mean(x)
    std_dev = np.std(x)
    x_normalized = (x - mean) / std_dev
    return x_normalized

calculation of rotation matrix

In [5]:
def rotationMatrix(axis,deg):
  thta = math.radians(deg)
  c, s = np.cos(thta), np.sin(thta)
  Rx = np.array([[1,0,0],[0,c,s],[0,-s,c]])
  Ry = np.array([[c,0,-s],[0,1,0],[s,0,c]])
  Rz = np.array([[c,-s,0],[s,c,0],[0,0,1]])
  RxRy = np.matrix(Rx)*np.matrix(Ry)
  RyRz = np.matrix(Ry)*np.matrix(Rz)
  RxRz = np.matrix(Rx)*np.matrix(Rz)
  if axis=="xy":
    R = RxRy
  elif axis=="yz":
    R = RyRz
  elif axis=="xz":
    R = RxRz
  return R    

rotate dataset

In [6]:
def rotate(data,R):
  size = data.shape[1]
  reminder = size%3
  #print(size,reminder)

  #first triplate
  triplate = data[:,:3]
  merged_array = np.array(np.matrix(triplate)*np.matrix(R))
  for i in range(3,size-reminder,3):
    triplate = data[:,i:3+i]
    Rotated = np.array(np.matrix(triplate)*np.matrix(R))
    # Merge the arrays along the columns
    merged_array = np.concatenate((merged_array, Rotated), axis=1)

  #Rotate if any colums are left
  if reminder!=0:
    triplate = data[:,-3:]
    Rotated = np.array(np.matrix(triplate)*np.matrix(R))
    merged_array = np.concatenate((merged_array, Rotated[:,-1*(reminder):]), axis=1)
    #print(merged_array.shape)
  return merged_array

3d rotation

In [7]:
def _3drotation(D,S):
  n = D.shape[1]
  numOfTriplates = n//3
  triplatesList = []
  for i in range(0,numOfTriplates,3):
    triplatesList.append(D[:,i:i+3])
  triplatesList.append(D[:,-3:])
  triplates = np.array(triplatesList)
  Veq=0
  for q in ["xy", "yz", "xz"]:
    thetaq = [_ for _ in range(361)]
    thetatemp = thetaq
    for theta in thetaq:
      Rq = rotationMatrix(q,theta)
      i=0
      for triplate in triplates:
        dq = np.array(np.matrix(triplate)*np.matrix(Rq))
        v1 = (np.var(triplate[:,0]-dq[:,0]))
        v2 = (np.var(triplate[:,1]-dq[:,1]))
        v3 = (np.var(triplate[:,2]-dq[:,2]))
        if v1<S[i][0] or v2<S[i][1] or v3<S[i][2]:
          if theta in thetatemp:
            thetatemp.remove(theta)
        i+=1
    alpha = random.choice(thetatemp)
    Deq=rotate(D,rotationMatrix(q,alpha))
    V = np.var(Deq)
    #rint(f"Variance of {q} axis is {V} " )
    if Veq<V:
      Veq = V
      Ddash = Deq
  return Ddash

#implimentation

Dataset

In [8]:
#df = pd.read_csv('/content/4. for_clasification.csv')
df = pd.read_csv('data.csv')
df.head()

,Class,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,1,67.0,68.0,73.0,78.0,65.0,63.0,67.0,60.0,63.0,...,61.0,56.0,76.0,75.0,74.0,77.0,76.0,74.0,59.0,68.0
1,1,75.0,74.0,71.0,71.0,62.0,58.0,70.0,64.0,71.0,...,66.0,62.0,68.0,69.0,69.0,66.0,64.0,58.0,57.0,52.0
2,1,83.0,64.0,66.0,67.0,67.0,74.0,74.0,72.0,64.0,...,67.0,64.0,69.0,63.0,68.0,54.0,65.0,64.0,43.0,42.0
3,1,72.0,66.0,65.0,65.0,64.0,61.0,71.0,78.0,73.0,...,69.0,68.0,68.0,63.0,71.0,72.0,65.0,63.0,58.0,60.0
4,1,62.0,60.0,69.0,61.0,63.0,63.0,70.0,68.0,70.0,...,66.0,66.0,58.0,56.0,72.0,73.0,71.0,64.0,49.0,42.0


In [9]:
label = df['Class']
df = df.drop(columns=['Class'])

In [10]:
k = int(input("Enter K: "))
A = df.to_numpy()
svdDf = svdd(A,k)
  
for i in range(svdDf.shape[1]):
  svdDf[:,i] = minmax_normalization(svdDf[:,i])

S = [[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4],[.4,.4,.4],[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4],[.4,.4,.4],[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4],[.4,.4,.4],[1,1,1],[.5,.5,.5],[.5,.5,.5],[.4,.4,.4]]
rotatedData = _3drotation(svdDf,S)
df = pd.DataFrame(rotatedData)
df['Class'] = np.array(list(label))
df.to_csv("5. SVD+3dRT.csv",index=False)